In [ ]:
import sys
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install pandas

In [ ]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species")
fig.show()

In [ ]:
import requests

bearer = "<set token here>"

URL = "https://dev1.ms.dev.symphony.com/api/dealer/v1/markets/search?limit=500"
searchParams = {
    #"keywords": [],
    #"startDateTime": "2021-05-06T10:06:06.916685Z",
    #"endDateTime": "2021-08-30T10:06:06.916685Z"
}

r = requests.post(url = URL, json = searchParams, headers = {'Authorization': bearer})

responseData = r.json()
print(responseData)

In [ ]:
import pandas as pd

dataset = dict()
for market in responseData["markets"]:
    instrument = market["legs"][0]["instrument"]
    currencyPair = instrument["baseCurrency"] + instrument["quoteCurrency"]
    tenor = instrument["tenor"]
    side = market["side"]

    foundPair = dataset.get(currencyPair, {})
    foundPair[tenor] = foundPair.get(tenor, 0) + 1
    dataset[currencyPair] = foundPair

currencypairs = []
tenors = []
counts = []

for currencyPair, data in dataset.items():
    for tenor, count in data.items():
        currencypairs.append(currencyPair)
        tenors.append(tenor)
        counts.append(count)

df = pd.DataFrame(dict(currencypair=currencypairs, tenor=tenors, count=counts))

fig = px.treemap(df, path=[px.Constant("all"), 'currencypair', 'tenor'], values='count')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()